In [1]:
import attn 
import torch
import numpy as np
torch.set_printoptions(threshold=np.inf)

In [21]:
LAM = attn.LocalAttention(
    neigh_size = 2,
    splits = 4,
    device = 'cpu',
    output_attention=True,
).eval()



In [22]:
attention = attn.AttentionLayer(
    attention = LAM,
    window_size=200,
    d_model = 64,
    n_heads = 4,
    d_k = 64,
    d_v = 64,
    mix = False
).eval()



In [23]:
Q = torch.randn(5, 6, 64).float()   # (batch_size, seq_length, d_model)
K = torch.randn(5, 6, 64).float()   # (batch_size, seq_length, d_model)
V = torch.randn(5, 6, 64).float()   # (batch_size, seq_length, d_model)

output, A = attention(Q, K, V)
print(output[0].shape)

torch.Size([6, 64])


In [24]:
Q_split, K_split, V_split, S, S_masked, A, V_split, output_refined = attention(Q, K, V, debugging=True)

In [25]:
Q_split.shape

torch.Size([3, 5, 2, 4, 16])

In [26]:
K_split.shape   

torch.Size([3, 5, 3, 4, 16])

In [34]:
A[0,0,0]

tensor([[0.0000, 1.0000, 0.0000],
        [0.0000, 0.4793, 0.5207]], grad_fn=<SelectBackward0>)

In [35]:
A.shape

torch.Size([3, 5, 4, 2, 3])

In [39]:
A_full = np.zeros((4, 5, 6, 6))

for slice in range(3):
    if slice == 0:
        i_idx = attention.inner_attention.split_indexes_Q[slice]
        j_idx = attention.inner_attention.split_indexes_KV[slice, 2-1:]
        A_slice = A[slice, :, :, :, 2-1:]
    else:
        i_idx = attention.inner_attention.split_indexes_Q[slice]
        j_idx = attention.inner_attention.split_indexes_KV[slice]
        A_slice = A[slice, :, :, :, :]

    A_full[:, :, i_idx, j_idx] = A_slice.detach().numpy()


ValueError: shape mismatch: value array of shape (5,4,2,2) could not be broadcast to indexing result of shape (4,5,2)

In [40]:
i_idx

tensor([0, 1])

In [41]:
j_idx

tensor([0, 1])

In [ ]:
slice = 5
batch = 2
head = 3
query = 1

np.argwhere(A_np[slice, batch, head, query] != 0).shape

In [ ]:
np.argwhere(A_np[slice, batch, head, query] != 0)

In [ ]:
np.argwhere(K_split[slice, batch, :, head, 1] != 0)

In [ ]:
np.argwhere(Q_split[slice, batch, :, head, 1] != 0)

In [ ]:
Q_split[0, batch, :, head, :].shape 

In [ ]:
K_split[0, batch, :, head, :].shape 

In [ ]:
Q = torch.randn(5, 200, 64).float()   # (batch_size, seq_length, d_model)
K = torch.randn(5, 200, 64).float()   # (batch_size, seq_length, d_model)
V = torch.randn(5, 200, 64).float()   # (batch_size, seq_length, d_model)

output = attention(Q, K, V)
print(output[0].shape)

In [ ]:
Q = torch.randn(5, 600, 64).float()   # (batch_size, seq_length, d_model)
K = torch.randn(5, 600, 64).float()   # (batch_size, seq_length, d_model)
V = torch.randn(5, 600, 64).float()   # (batch_size, seq_length, d_model)

output = attention(Q, K, V)
print(output[0].shape)

In [ ]:
import sys
torch.full((5, 5), -sys.maxsize)

In [ ]:
window_size = 10
neigh_size = 3
full_mask = torch.tril(torch.ones((window_size,window_size), dtype=torch.double)) - torch.tril(torch.ones((window_size,window_size), dtype=torch.double),diagonal=-neigh_size)


In [ ]:
full_mask

In [ ]:
torch.tril(torch.ones((window_size,window_size), dtype=torch.double))

In [ ]:
torch.tril(torch.ones((window_size,window_size), dtype=torch.double),diagonal=-neigh_size)

In [ ]:
matrix = torch.rand((4,4))

In [ ]:
print(matrix)

In [ ]:
torch.einsum('ii, jj -> ij', matrix, matrix)

In [ ]:
torch.einsum('ii', matrix)

In [ ]:
"""
B S L H E @ B S T H E -> B S L H T
            B S E H T

"""

In [ ]:
torch.arange(6).shape

In [ ]:
split_indexes = []
B = 8
L = 100
H = 4
E = 3
neigh_size = 5
l1 = neigh_size
splits = L // neigh_size

for j in torch.arange(0, splits):
    for i in torch.arange(0, l1):
        split_indexes.append(i+l1*j)
split_indexes_Q = torch.tensor(split_indexes).reshape(splits, l1)

In [ ]:
# Matrix SI such that for split i, the block i of K has the rows with index in SI[i,:]
split_indexes = [] 
split_indexes_KV_end = None
for j in torch.arange(0, splits):
    for i in torch.arange(0, l1+neigh_size):
        split_indexes.append(i+l1*j-neigh_size)
split_indexes_KV = torch.tensor(split_indexes).reshape(splits, l1+neigh_size)
split_indexes_KV[0] = torch.cat((torch.zeros(neigh_size), torch.arange(0, l1)), 0)

In [ ]:
queries = torch.arange(0, B*L*H*E).reshape(B, L, H, E)
Q_expand = queries.unsqueeze(0).expand(splits, B, L, H, E).permute(1,0,2,3,4)
Q_split = Q_expand[:, torch.arange(splits).unsqueeze(1), split_indexes_Q, :].transpose(0,1)


In [ ]:
keys = queries.clone()

# Split keys (here we need more rows per block to capture the attention scores of queries and those elements of keys at positions j l1 - neigh_size to j l1.
K_expand = keys.unsqueeze(0).expand(splits, B, L, H, E).permute(1,0,2,3,4)
K_split = K_expand[:, torch.arange(splits).unsqueeze(1), split_indexes_KV, :].transpose(0,1)
K_split[0,:,:neigh_size,:,:] = 0

In [ ]:
Q_split.shape

In [ ]:
K_split.shape

In [ ]:
A.shape

In [ ]:
A[2, 1, 2]

In [ ]:
print("Q_split shape:", Q_split.shape)
print("K_split shape:", K_split.shape)
S = torch.einsum("sblhe,sbthe->sbhlt", Q_split, K_split)
print("S shape:", S.shape)

In [ ]:
A = torch.softmax(S.float(), dim=-1)


In [ ]:
torch.cat(A, 2)

In [ ]:
A.shape